# Training

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
np.random.seed(7777)
tf.random.set_seed(7777)

In [3]:
class Cifar10DataLoader():
    def __init__(self):
        # data load
        (self.train_x, self.train_y), \
            (self.test_x, self.test_y) = tf.keras.datasets.cifar10.load_data()
        self.input_shape = self.train_x.shape[1:]

    def scale(self, x):

        return (x / 255.0).astype(np.float32)

    def preprocess_dataset(self, dataset):

        (feature, target) = dataset

        # scaling #
        scaled_x = np.array([self.scale(x) for x in feature])

        # label encoding #
        ohe_y = np.array([tf.keras.utils.to_categorical(
            y, num_classes=10) for y in target])
        
        return scaled_x, ohe_y.squeeze(1)

    def get_train_dataset(self):
        return self.preprocess_dataset((self.train_x, self.train_y))

    def get_test_dataset(self):
        return self.preprocess_dataset((self.test_x, self.test_y))

cifar10_loader = Cifar10DataLoader()
train_x, train_y = cifar10_loader.get_train_dataset()

print(train_x.shape, train_x.dtype)
print(train_y.shape, train_y.dtype)

test_x, test_y = cifar10_loader.get_test_dataset()

print(test_x.shape, test_x.dtype)
print(test_y.shape, test_y.dtype)

(50000, 32, 32, 3) float32
(50000, 10) float32
(10000, 32, 32, 3) float32
(10000, 10) float32


In [4]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Add

def build_resnet(input_shape):
    inputs = Input(input_shape)

    net = Conv2D(32, kernel_size=3, strides=2,
                 padding='same', activation='relu')(inputs)
    net = MaxPool2D()(net)
    
    net1 = Conv2D(64, kernel_size=1, padding='same', activation='relu')(net)
    net2 = Conv2D(64, kernel_size=3, padding='same', activation='relu')(net1)
    net3 = Conv2D(64, kernel_size=1, padding='same', activation='relu')(net2)
    
    net1_1 = Conv2D(64, kernel_size=1, padding='same')(net)
    net = Add()([net1_1, net3])
    
    net1 = Conv2D(64, kernel_size=1, padding='same', activation='relu')(net)
    net2 = Conv2D(64, kernel_size=3, padding='same', activation='relu')(net1)
    net3 = Conv2D(64, kernel_size=1, padding='same', activation='relu')(net2)
    
    net = Add()([net, net3])
    
    net = MaxPool2D()(net)
    
    net = Flatten()(net)
    net = Dense(10, activation="softmax")(net)

    model = tf.keras.Model(inputs=inputs, outputs=net, name='resnet')
    
    return model

model = build_resnet((32, 32, 3))
model.summary()

Model: "resnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 16, 16, 32)   896         ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 8, 8, 32)     0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 8, 8, 64)     2112        ['max_pooling2d[0][0]']          
                                                                                             

## fit 함수 사용

$$
\Large{
model.compile() \rightarrow model.fit()
}
$$

#### compile의 입력값

- optimizer='rmsprop' : Optimizer 
- loss=None : Loss function
- metrics=None : Metrics
- loss_weights=None : loss가 여러 개인 경우 각 로스마다 다르게 중요도를 설정 할 수 있다. 


In [5]:
lr = 0.03
opt = tf.keras.optimizers.Adagrad(lr)
loss = tf.keras.losses.categorical_crossentropy

In [6]:
model.compile(optimizer=opt, loss = loss, metrics=['accuracy'])

In [7]:
# 내가 개발한 loss를 써보고싶다
def custom_loss(y_true, y_pred):    
                # 정답 , 예측치
    return tf.reduce_mean(tf.square(y_true - y_pred))  # mse

In [8]:
model.compile(optimizer=opt, loss=custom_loss, metrics=['accuracy'])

In [ ]:
# loss를 여러개 사용하고 싶을 때,
model.compile(optimizer=opt, loss = [loss, custom_loss], metrics =['accuracy'])

In [10]:
# loss weight
model.compile(optimizer=opt, loss=[loss, custom_loss], metrics=['accuracy'], loss_weights=[0.9, 0.1])
                                                                                            #1번 loss
                                                                                                #2번 loss

                            # 최종 결과 loss는 두 loss를 더한 값인데, loss_weights 만큼 곱해진 값들을 더한 결과를 반환한다.

In [11]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
                                # 이름을 넣어도 된다.

In [12]:
acc = tf.keras.metrics.Accuracy()
auc = tf.keras.metrics.AUC()


In [13]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=[acc,auc])

### fit의 입력값

- x=None 
- y=None
- batch_size=None
- epochs=1
- verbose='auto' : 학습과정 출력문의 모드
- callbacks=None : Callback 함수
- validation_split=0.0 : 입력데이터의 일정 부분(%)을 Validation 용 데이터로 사용함
- validation_data=None : Validation 용 데이터 
- shuffle=True : 입력값을 Epoch 마다 섞는다. // True가 기본값임
- class_weight=None : 클래스 별로 다른 중요도를 설정한다. 
- ...

In [14]:
hist = model.fit(
    train_x,
    train_y,
    epochs=1,
    batch_size = 64,
    verbose=1
)

782/782 [==============================] - 17s 19ms/step - loss: 1.7339 - accuracy: 0.0000e+00 - auc: 0.8172


### Callback 함수 활용하기 

 - Callback 함수를 활용하면, fit() 함수가 돌아가는 와중에도 특정한 주기로 원하는 코드를 실행 시킬 수 있음. 
 
 ex> 학습이 진행되면, Learning rate를 점점 줄여서 더 세밀하게 모델의 웨이트가 조정 될 수 있도록 하고 싶다. 

In [17]:
def scheduler(epoch, lr):
    if epoch > 10:
        return lr * (0.9**(epoch - 10))
    else:
        return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)
                                                                    # 상황을 보고싶다면 verbose=1

In [18]:
hist = model.fit(
    train_x,
    train_y,
    epochs=1,
    batch_size = 64,
    validation_split=0.3,
    verbose=1,
    callbacks=(lr_scheduler)
)


Epoch 1: LearningRateScheduler setting learning rate to 0.029999999329447746.
547/547 [==============================] - 18s 33ms/step - loss: 1.3909 - accuracy: 0.0000e+00 - auc: 0.8892 - val_loss: 1.3326 - val_accuracy: 0.0000e+00 - val_auc: 0.8981 - lr: 0.0300


In [ ]:
tf.keras.callbacks.EarlyStopping
                    # epoch의 값이 어느정도 올랐을 때,
                    # loss 값이 계속 비슷하다면 그즘에서 멈추는 작업을 한다.